# Setup 

In [ ]:
!rm -r /kaggle/working/*

In [ ]:
!cat /etc/os-release
!python -V

import os
import torch

print(f"GPUs ({torch.cuda.device_count()}):")
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))
os.environ['CPU_COUNT'] = str(os.cpu_count())

!echo $CPU_COUNT

In [ ]:
import os
from kaggle_secrets import UserSecretsClient
password = UserSecretsClient().get_secret("gh_token")
os.environ['GITHUB_AUTH'] = 'marcodiri:' + password
!git clone https://$GITHUB_AUTH@github.com/marcodiri/s2s-contrastive-text-recognition.git ~/s2s

In [ ]:
!mkdir -p ~/s2s/datasets/iam/words
!tar zxf /kaggle/input/iamdatasethtrwords/words.tgz -C ~/s2s/datasets/iam/words

In [ ]:
%cd ~/s2s/datasets
!python process_iam.py

In [ ]:
!pip install -q fire
%cd ~/s2s/datasets
!python create_lmdb_dataset.py \
  --inputPath iam_train/ --gtFile iam_train/gt.txt --outputPath lmdb_train/IAM
!python create_lmdb_dataset.py \
  --inputPath iam_train5pc/ --gtFile iam_train5pc/gt.txt --outputPath lmdb_train5pc/IAM
!python create_lmdb_dataset.py \
  --inputPath iam_val/ --gtFile iam_val/gt.txt --outputPath lmdb_val/IAM
!python create_lmdb_dataset.py \
  --inputPath iam_test/ --gtFile iam_test/gt.txt --outputPath lmdb_test/IAM

# Training

In [ ]:
%cd ~/s2s
!pip install -q matplotlib lmdb torch==1.13.1 pytorch-lightning==1.8.6 torchvision pandas kornia==0.6.8 torchmetrics==0.11.0 natsort importlib-metadata

### Decoder-training

In [ ]:
%cd ~/s2s
# download pre-trained encoder None-VGG-BiLSTM-None-Seed1111/epoch=32-step=8250-train_loss=0.01.ckpt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XSCpGvb2BqkZCcJW8iUoriLWFwaxPHlK' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XSCpGvb2BqkZCcJW8iUoriLWFwaxPHlK" -O 'encoder.ckpt' && rm -rf /tmp/cookies.txt

# download pre-trained encoder None-VGG-BiLSTM-WindowToInstance-Seed1111/epoch=41-step=2100-train_loss=0.03.ckpt
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=17rwL_EIpyE4WZNv_FnXghNlFNzjvJy0f' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=17rwL_EIpyE4WZNv_FnXghNlFNzjvJy0f" -O 'encoder.ckpt' && rm -rf /tmp/cookies.txt

In [ ]:
%cd ~/s2s
!python train_decoder.py \
--exp_name 'decoder_eval-100pc-NoInstanceMapping' \
--pretrained_encoder 'encoder.ckpt' \
--train_data ~/s2s/datasets/lmdb_train --select_data '/' \
--valid_data ~/s2s/datasets/lmdb_val --sensitive \
--num_iter 50000 --val_interval 4 --batch_size 253 \
--log_every_n_steps 38 \
--workers 2 --save_dir '/kaggle/working/saved_models/' \
--lr 10 --plateau 0.1 --patience 5 --earlystopping \
--Transformation None --FeatureExtraction VGG --SequenceModeling BiLSTM --Prediction Attn